# MAE evaluation

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import mean_absolute_error

In [ ]:
BASE_PATH = '/root/work/datasets/artwork_sequence/'
CONFIG_PATH = '/root/work/artwork_sequence/train_test_configuration'
RESULT_PATH = '/root/work/artwork_sequence/predicted_tours/generated_sequence_based_previous_most_similar'

## Load data

**Load all metadata**

In [ ]:
df_all_metadata = pd.read_csv(os.path.join(BASE_PATH, 'all_metadata.csv'))
all_data_matrix = np.load(os.path.join(BASE_PATH, 'all_code_matrix.npy' ))

In [ ]:
print(df_all_metadata.shape)
print(all_data_matrix.shape)

**Load tours**

In [ ]:
museum_sequence_path = {
    'x_train' : os.path.join(CONFIG_PATH, 'X_train.csv'),
    'x_test' : os.path.join(CONFIG_PATH, 'X_test.csv'),
    'x_train_matrix' : os.path.join(CONFIG_PATH, 'X_train_matrix.npy'),
    'x_test_matrix' : os.path.join(CONFIG_PATH, 'X_test_matrix.npy')
}
museum_sequence_path

In [ ]:
df_x_train = pd.read_csv(museum_sequence_path['x_train'], index_col=0)
df_x_test = pd.read_csv(museum_sequence_path['x_test'], index_col=0)
x_train_matrix = np.load(museum_sequence_path['x_train_matrix'])
x_test_matrix = np.load(museum_sequence_path['x_test_matrix'])
df_x_train.head()

In [ ]:
tour_list = df_x_test['tour_path'].values

## Load model

In [ ]:
from Sequence_generator_based_previous_most_similar import Sequence_generator_based_previous_most_similar

In [ ]:
def get_model(X_tour, window_size, all_data_matrix, df_X_tour,df_all_metadata):

    seq_generator_most_sim = Sequence_generator_based_previous_most_similar( 
        X_tour=X_tour, 
        window_size=window_size, 
        all_data_matrix=all_data_matrix, 
        df_X_tour=df_X_tour, 
        df_all_metadata=df_all_metadata )
    
    return seq_generator_most_sim

## Compute MAE

In [ ]:
df_mae = pd.DataFrame({ 'model' : [],
                         'tour' : [],
                         'mae' : []})
df_mae

In [ ]:
index = 0
window_size = 3

for tour in tour_list:
    #Get tour
    images_path = os.path.join(tour, 'images')
    tour_length = len(os.listdir(images_path))
    X_tour = x_test_matrix[index: index + tour_length]
    
    metadata_path = os.path.join(tour, 'metadata.csv')
    df_X_tour = pd.read_csv(metadata_path)
    
    index += tour_length 
    
    #Define model
    model = get_model(
        X_tour=X_tour, 
        all_data_matrix=all_data_matrix, 
        df_X_tour=df_X_tour, 
        df_all_metadata=df_all_metadata, 
        window_size=window_size)
    
    df_predicted_tour = model.predict_tour()
    
    #Compute MAE
    forecast_matrix = model.get_predicted_tour_matrix()
    mae = np.mean(mean_absolute_error( X_tour[window_size:,:].T, forecast_matrix.T, multioutput='raw_values' ))
    
    #Save tour
    tour_name = tour.split('/')[-1]
    df_predicted_tour.to_csv(os.path.join(RESULT_PATH, 'predicted_' + tour_name + '.csv'), index=False)
    
    #Save MAE
    df_mae = df_mae.append(
        {'model' : model.get_name(),
         'tour': tour_name,
         'mae': mae
        }, 
       ignore_index=True)

df_mae

In [ ]:
df_mae['mae'].mean()